In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
demo = pd.read_csv('DEMO.csv')
mclass = pd.read_csv('MCLASS.csv')
txkea = pd.read_csv('TXKEA.csv')

/var/folders/9m/3m9g9yzs77j2m_79z47fjdr00000gq/T/ipykernel_59067/2206552592.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  demo = pd.read_csv('DEMO.csv')


In [3]:
demo

,student_id,district_id,ethnicity,eco,el,spec_ed
0,97840593,798403,Black or African American,YES,NO,NO
1,885938600,53405,White,YES,NO,NO
2,871944576,798403,Black or African American,YES,NO,NO
3,818725252,53405,White,NO,NO,NO
4,702015143,800409,White,YES,NO,YES
...,...,...,...,...,...,...
174343,S75470395,795107,Hispanic/Latino,NO,NO,NO
174344,S75470364,795107,White,NO,NO,NO
174345,S75470354,795107,Two or more races,YES,NO,NO
174346,S75470343,795107,White,NO,NO,NO


In [45]:
demo.isna().sum()

student_id     0
district_id    0
ethnicity      0
eco            0
el             0
spec_ed        0
dtype: int64

In [4]:
mclass

,student_id,district_id,school_id,assessment_edition,composite_level,composite_score
0,8878547139,806405.0,806405802,DIBELS 8th Edition,At Benchmark,306.0
1,8878132753,818408.0,818408807,DIBELS 8th Edition,Below Benchmark,291.0
2,8877357966,NaN,NaN,DIBELS 8th Edition,At Benchmark,314.0
3,8877359986,820405.0,820405805,DIBELS 8th Edition,At Benchmark,326.0
4,8877961413,820407.0,820407871,DIBELS 8th Edition,At Benchmark,308.0
...,...,...,...,...,...,...
63263,4176193481,778409.0,778409807,DIBELS 8th Edition,At Benchmark,326.0
63264,4137115834,801102.0,801102842,DIBELS 8th Edition,Well Below Benchmark,245.0
63265,4168441274,858404.0,858404807,DIBELS 8th Edition,Below Benchmark,290.0
63266,4148865139,858404.0,858404807,DIBELS 8th Edition,Well Below Benchmark,261.0


In [46]:
mclass.isna().sum()
#check for nulls
#keep all rows because score is not very relevant given the information from the senario

student_id               0
district_id           6240
school_id             6592
assessment_edition       0
composite_level          0
composite_score       7144
dtype: int64

In [5]:
txkea

,district_id,student_id,language,lit_screening_benchmark,lit_screening_score,date
0,70408,8878861576,English,Monitor,15,9/15/2020
1,808486,8878825752,English,On-Track,29,11/30/2020
2,801404,8878799239,English,Support,14,10/12/2020
3,808488,8878794629,English,On-Track,35,11/6/2020
4,808486,8878745384,English,On-Track,23,10/16/2020
...,...,...,...,...,...,...
112043,770406,4414214257,English,On-Track,22,1/29/2021
112044,76408,4414156744,English,On-Track,23,9/11/2020
112045,86485,4414112931,English,Support,6,10/15/2020
112046,856408,4414484419,English,On-Track,24,9/30/2020


In [47]:
txkea.isna().sum()

district_id                0
student_id                 0
language                   0
lit_screening_benchmark    0
lit_screening_score        0
date                       0
dtype: int64

In [6]:
from pandasql import sqldf

In [39]:
query = 'select d.*, m.composite_level as mclass_lvl, m.composite_score as mclass_score, \
t.lit_screening_benchmark as txkea_level, t.lit_screening_score as txkea_score \
from demo as d \
left join mclass as m on d.student_id = m.student_id \
left join txkea as t on d.student_id = t.student_id \
where m.assessment_edition = \'DIBELS 8th Edition\' \
or t.language = \'English\';'

# this query makes a df of all students who took either the mclass or txkea tests in English, their scores,
# and some of their demographics

In [35]:
result = sqldf(query, locals())

In [36]:
result.head(1000)

,student_id,district_id,ethnicity,eco,el,spec_ed,mclass_lvl,mclass_score,txkea_level,txkea_score
0,885938600,53405,White,YES,NO,NO,None,NaN,On-Track,37.0
1,871944576,798403,Black or African American,YES,NO,NO,None,NaN,On-Track,21.0
2,818725252,53405,White,NO,NO,NO,None,NaN,On-Track,42.0
3,702015143,800409,White,YES,NO,YES,None,NaN,On-Track,19.0
4,717968813,48403,Two or more races,YES,NO,NO,None,NaN,On-Track,21.0
5,712586284,795107,White,YES,NO,NO,None,NaN,On-Track,24.0
6,748531398,795107,White,NO,NO,NO,None,NaN,On-Track,42.0
7,307161742,795107,Hispanic/Latino,NO,NO,NO,None,NaN,Support,11.0
8,370225802,795107,Two or more races,YES,NO,NO,None,NaN,On-Track,31.0
9,373122722,798403,White,YES,NO,NO,None,NaN,On-Track,20.0


In [37]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150257 entries, 0 to 150256
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   student_id    150257 non-null  object 
 1   district_id   150257 non-null  int64  
 2   ethnicity     150257 non-null  object 
 3   eco           150257 non-null  object 
 4   el            150257 non-null  object 
 5   spec_ed       150257 non-null  object 
 6   mclass_lvl    56155 non-null   object 
 7   mclass_score  56149 non-null   float64
 8   txkea_level   94573 non-null   object 
 9   txkea_score   94573 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 11.5+ MB


In [38]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174348 entries, 0 to 174347
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   student_id   174348 non-null  object
 1   district_id  174348 non-null  int64 
 2   ethnicity    174348 non-null  object
 3   eco          174348 non-null  object
 4   el           174348 non-null  object
 5   spec_ed      174348 non-null  object
dtypes: int64(1), object(5)
memory usage: 8.0+ MB


# Data exploration and Analysis

## How many students of each demographic took each assessment?

In [48]:
result.head()

,student_id,district_id,ethnicity,eco,el,spec_ed,mclass_lvl,mclass_score,txkea_level,txkea_score
0,885938600,53405,White,YES,NO,NO,None,NaN,On-Track,37.0
1,871944576,798403,Black or African American,YES,NO,NO,None,NaN,On-Track,21.0
2,818725252,53405,White,NO,NO,NO,None,NaN,On-Track,42.0
3,702015143,800409,White,YES,NO,YES,None,NaN,On-Track,19.0
4,717968813,48403,Two or more races,YES,NO,NO,None,NaN,On-Track,21.0


In [ ]:
eco = 